In [1]:
with open('input') as f:
    intcode = [int(n) for n in f.read().split(',')]
intcode[:10]

[3, 8, 1005, 8, 342, 1106, 0, 11, 0, 0]

In [2]:
from collections import defaultdict

def process_intcode(intcode, start_white=False):
    intcode = {i: instruction for i, instruction in enumerate(intcode)}
    map = defaultdict(int)
    pos = (0, 0)
    visited = {pos}
    if start_white:
        map[pos] = 1
    direction_delta = (0, -1)
    output_num = 0
    relative_base = 0
    i = 0
    while True:
        initial = str(intcode.get(i, 0)).rjust(5, '0')
        a, b, c, *opcode = initial
        opcode = int(''.join(opcode))
        assert opcode in (1, 2, 3, 4, 5, 6, 7, 8, 9, 99), opcode
        if opcode == 99:
            return map, visited
        
        if c == '0':
            p1 = intcode.get(i + 1, 0)
        elif c == '1':
            p1 = i + 1 
        elif c == '2':
            p1 = intcode.get(i + 1, 0) + relative_base
            
        if b == '0':
            p2 = intcode.get(i + 2, 0)
        elif b == '1':
            p2 = i + 2
        elif b == '2':
            p2 = intcode.get(i + 2, 0) + relative_base
            
        if a == '0':
            p3 = intcode.get(i + 3, 0)
        elif a == '1':
            p3 = i + 3
        elif a == '2':
            p3 = intcode.get(i + 3, 0) + relative_base
            
        if opcode == 1:
            intcode[p3] = intcode.get(p1, 0) + intcode.get(p2, 0)
            i += 4
        elif opcode == 2:
            intcode[p3] = intcode.get(p1, 0) * intcode.get(p2, 0)
            i += 4
        elif opcode == 3:
            intcode[p1] = map[pos]
            i += 2
        elif opcode == 4:
            output_value = intcode.get(p1, 0)
            if output_num == 0:
                visited.add(pos)
                map[pos] = output_value
                output_num = 1
            else:
                direction_delta = {
                    ((0, -1), 0): (-1, 0),
                    ((0, -1), 1): (1, 0),
                    ((1, 0), 0): (0, -1),
                    ((1, 0), 1): (0, 1),
                    ((0, 1), 0): (1, 0),
                    ((0, 1), 1): (-1, 0),
                    ((-1, 0), 0): (0, 1),
                    ((-1, 0), 1): (0, -1),
                }[(direction_delta, output_value)]
                pos = (pos[0] + direction_delta[0], pos[1] + direction_delta[1])
                output_num = 0
            i += 2
        elif opcode == 5:
            if intcode.get(p1, 0) != 0:
                i = intcode.get(p2, 0)
            else:
                i += 3
        elif opcode == 6:
            if intcode.get(p1, 0) == 0:
                i = intcode.get(p2, 0)
            else:
                i += 3
        elif opcode == 7:
            intcode[p3] = 1 if intcode.get(p1, 0) < intcode.get(p2, 0) else 0
            i += 4
        elif opcode == 8:
            intcode[p3] = 1 if intcode.get(p1, 0) == intcode.get(p2, 0) else 0
            i += 4
        elif opcode == 9:
            relative_base += intcode.get(p1, 0)
            i += 2

In [3]:
map, visited = process_intcode(intcode)
print("Part 1:")
print(len(visited))

Part 1:
2172


In [4]:
def display_map(map):
    min_x = min(p[0] for p in map)
    max_x = max(p[0] for p in map)
    min_y = min(p[1] for p in map)
    max_y = max(p[1] for p in map)

    for y in range(min_y, max_y + 1):
        for x in range(min_x, max_x + 1):
            print('█' if map[(x, y)] else ' ', end='')
        print()

In [5]:
map, visited = process_intcode(intcode, start_white=True)
print("Part 2:")
print()
display_map(map)

Part 2:

   ██ ████ █    ████ ████  ██  █  █ ███    
    █ █    █    █    █    █  █ █  █ █  █   
    █ ███  █    ███  ███  █    ████ █  █   
    █ █    █    █    █    █ ██ █  █ ███    
 █  █ █    █    █    █    █  █ █  █ █      
  ██  ████ ████ ████ █     ███ █  █ █      
